In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from state import get_state, queue
import os
import traci

In [3]:
from dotenv import load_dotenv
load_dotenv()


sumo_bin = os.getenv("SUMO")
sumo_gui_bin = os.getenv("SUMO-GUI")
simulConfig = os.getenv("SIMUL-CONFIG")

In [4]:
import tensorflow as tf
import numpy as np
from collections import deque

In [13]:

class DuelingDQN(tf.keras.Model):
    def __init__(self, input_shape, n_outputs):
        super(DuelingDQN, self).__init__()

        # Couche d'entrée commune
        self.shared_layers = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation="relu", input_shape=input_shape),
            tf.keras.layers.Dense(128, activation="relu"),
        ])

        # **Stream Valeur** V(s)
        self.value_stream = tf.keras.layers.Dense(1)  # Une seule sortie : la valeur de l'état

        # **Stream Avantage** A(s, a)
        self.advantage_stream = tf.keras.layers.Dense(n_outputs)  # Une sortie par action

    def call(self, inputs):
        x = self.shared_layers(inputs)

        V = self.value_stream(x)  # Calcul de la valeur de l’état
        A = self.advantage_stream(x)  # Calcul des avantages

        # Normalisation de A pour éviter le biais
        Q = V + (A - tf.reduce_mean(A, axis=1, keepdims=True))

        return Q


In [14]:
tf.random.set_seed(42)
input_shape = [48]  # Taille de l'état (exemple)
n_outputs = 4  # Nombre d'actions possibles

# Création du modèle principal et du réseau cible
model_action = DuelingDQN(input_shape, n_outputs)
target = DuelingDQN(input_shape, n_outputs)
target.set_weights(model_action.get_weights())  # Copie des poids initiaux

In [15]:
def epsilon_greedy_policy(state, epsilon=0):
    if np.random.rand() < epsilon:
        return np.random.randint(n_outputs)  # random action
    else:
        Q_values = model_action.predict(state[np.newaxis], verbose=0)[0]
        print(f"state : {state}")
        print(f"Q_values : {Q_values}")
        return Q_values.argmax()  # optimal action according to the DQN

In [16]:
def sample_experiences(batch_size):
    indices = np.random.randint(len(replay_buffer), size=batch_size)
    batch = [replay_buffer[index] for index in indices]
    states, actions, rewards, next_states = [
        np.array([experience[field_index] for experience in batch])
        for field_index in range(4)
    ]
    return states, actions, rewards, next_states

In [17]:
batch_size = 32
discount_factor = 0.5
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss_fn = tf.keras.losses.MeanSquaredError()


def training_step(batch_size):
    experiences = sample_experiences(batch_size)
    states, actions, rewards, next_states = experiences  # a changer
        #################### CHANGED SECTION ####################
    next_Q_values = model_action.predict(next_states, verbose=0)  # ≠ target.predict()
    best_next_actions = next_Q_values.argmax(axis=1)
    next_mask = tf.one_hot(best_next_actions, n_outputs).numpy()
    max_next_Q_values = (target.predict(next_states, verbose=0) * next_mask
                        ).sum(axis=1)
    #########################################################
    next_Q_values = target.predict(next_states, verbose=0)
    max_next_Q_values = next_Q_values.max(axis=1)
    # runs = 1.0 - (dones | truncateds)  # episode is not done or truncated
    target_Q_values = rewards + discount_factor * max_next_Q_values
    target_Q_values = target_Q_values.reshape(-1, 1)
    mask = tf.one_hot(actions, n_outputs)
    with tf.GradientTape() as tape:
        all_Q_values = model_action(states)
        Q_values = tf.reduce_sum(all_Q_values * mask, axis=1, keepdims=True)
        loss = tf.reduce_mean(loss_fn(target_Q_values, Q_values))

    grads = tape.gradient(loss, model_action.trainable_variables)
    optimizer.apply_gradients(zip(grads, model_action.trainable_variables))

In [ ]:
sumoConfig3 = r"Traditional_traffic/traditional_traffic.sumo.cfg"

replay_buffer = deque(maxlen=10000)


sumoCmd = [sumo_bin, "-c",simulConfig,'--start','--no-warnings']
print(sumoCmd)
for episode in range(40):
    if traci.isLoaded():
        traci.close()
    traci.start(sumoCmd)
    lane_ids =  traci.lane.getIDList()
    # print(lane_ids[0])

    trafic_light_ids = traci.trafficlight.getIDList()

    # state = np.array(queue(lane_ids))
    state=np.array(get_state(lane_ids))
    action=-1
    # print(state)
    for step in range(130000): ## TO CHANGED
        epsilon = max(1 - episode+120 / 160, 0.01)

        if step%2000 == 0:
            #######################################################################
            next_state = np.array(get_state(lane_ids))
            reward = np.sum(state[:24])- np.sum(next_state[:24])
            replay_buffer.append((state, action, reward, next_state))
            #######################################################################

            state=next_state
            action = epsilon_greedy_policy(state, epsilon)
            traci.trafficlight.setPhase(trafic_light_ids[0],2*action)

            if len(replay_buffer) >= batch_size*10:
                training_step(batch_size)

        traci.simulationStep()

    if episode%5==0:
        target.set_weights(model_action.get_weights())
    print(f'episode : {episode}')
    traci.close()


['/Users/arseneclaustre/sumo/bin/sumo', '-c', 'Traditional_traffic/traditional_traffic.sumo.cfg', '--start', '--no-warnings']
 Retrying in 1 seconds


Step #1260.00 (0ms ?*RT. ?UPS, TraCI: 0ms, vehicles TOT 924 ACT 61 BUF 0)                  #180.00 (1ms ~= 10.00*RT, ~77000.00UPS, TraCI: 1ms, vehicles TOT 148 ACT 77 BUF 8)   0.00 (0ms ?*RT. ?UPS, TraCI: 2ms, vehicles TOT 280 ACT 65 BUF 8)                   (0ms ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 412 ACT 53 BUF 8)                  s ?*RT. ?UPS, TraCI: 1ms, vehicles TOT 559 ACT 70 BUF 5)                  RT. ?UPS, TraCI: 1ms, vehicles TOT 703 ACT 76 BUF 5)                   ?UPS, TraCI: 1ms, vehicles TOT 835 ACT 84 BUF 5)                 episode : 0
Step #1300.00 (0ms ?*RT. ?UPS, TraCI: 2ms, vehicles TOT 924 ACT 60 BUF 0)                 
 Retrying in 1 seconds


state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 3 2 2 2 4 2 2 3 3 2 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-14.45553  -10.549704 -10.3311   -10.484062]
state : [ 0  2  6  0  0  2  0  2  2  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0
  0  2 10  2  2  4  4  2  2  0  3  2  0  0  0  3  4  0  0  0  0  0  0  0]
Q_values : [ 0.6969881   0.88783586 -2.467606   -0.37102592]
Stepstate : [ 0  0  5  0  3  1  0  0  1  0  9  1  0  0  0  0  0  0  0  0  0  0  0  0 0)   
  0  0 10  0  4  2  4  0  2  0 13  2  2  2  1  4  0  0  2  3  0  2  0  0]
Q_values : [1.3208722  4.4934735  4.3333554  0.46226048]
state : [ 0  0  3  0  6  0  0  0  0  0 16  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  0  7  2  6  0  5  0  0  1 20  0  2  0  1  2  0  1  0  0  3  0  0  0]
Q_values : [ 5.9033813 -0.5551715 14.26734    2.647236 ]
state : [ 0  0 13  0  0  2  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  0 17  1  2  4  5  0  0  1  7  0  0  0  0  1  1  0  0  0  0  0  0  0]
Q_values : [-0.8799108   2.4646

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-4.6216574 -3.7940562 -3.988247  -3.8893688]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 3 2 2 2 4 2 2 3 3 2 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-14.45553  -10.549704 -10.3311   -10.484062]
state : [0 2 2 0 0 2 0 2 2 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 1 2 7 2 0 2 5 2 2 1 3 2 2
 0 0 2 2 0 2 0 0 1 2 0]
Q_values : [ 0.6371434   0.76693666 -2.5841815  -0.54121864]
state : [0 2 0 0 0 2 0 2 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 2 2 4 4 2 0 0 6 2 0
 0 0 3 0 8 0 0 0 0 0 1]
Q_values : [-6.0034847 -6.4176893 -4.258577  -2.8215158]
state : [0 2 2 0 2 0 0 2 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 6 4 3 0 4 2 0 1 9 0 2
 0 1 4 0 0 0 0 3 0 0 0]
Q_values : [-0.18282914 -0.88344353  1.737735   -2.5936043 ]
state : [ 0  2  6  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  2 10  1  1  2  5  2  0  1  3  0  0  0  0  3  5  4  0  0  0  0  2  0]
Q_values : [-1.39

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-4.6216574 -3.7940562 -3.988247  -3.8893688]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 3 2 2 2 4 2 2 3 3 2 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-14.45553  -10.549704 -10.3311   -10.484062]
state : [0 2 2 0 0 2 0 2 2 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 1 2 7 2 0 2 5 2 2 1 3 2 2
 0 0 2 2 0 2 0 0 1 2 0]
Q_values : [ 0.6371434   0.76693666 -2.5841815  -0.54121864]
state : [0 2 0 0 0 2 0 2 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 2 2 4 4 2 0 0 6 2 0
 0 0 3 0 8 0 0 0 0 0 1]
Q_values : [-6.0034847 -6.4176893 -4.258577  -2.8215158]
state : [0 2 2 0 2 0 0 2 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 6 4 3 0 4 2 0 1 9 0 2
 0 1 4 0 0 0 0 3 0 0 0]
Q_values : [-0.18282914 -0.88344353  1.737735   -2.5936043 ]
state : [ 0  2  6  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  2 10  1  1  2  5  2  0  1  3  0  0  0  0  3  5  4  0  0  0  0  2  0]
Q_values : [-1.39

...

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-5.1884165 -3.976118  -4.732705  -4.1579194]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 3 2 2 2 4 2 2 3 3 2 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-12.9722   -10.215965 -11.136139 -10.427279]
state : [0 2 0 0 2 2 0 2 0 0 3 2 0 0 0 0 0 0 0 0 0 0 0 0 1 2 3 2 2 2 6 2 0 1 7 2 2
 0 0 2 0 3 2 0 0 2 0 2]
Q_values : [1.4689276 1.519573  2.412034  2.5295773]
state : [ 0  2  2  0  2  0  0  2  0  0  6  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  2  6  2  4  2  4  2  0  0 10  0  0  0  1  4  0  1  0  0  1  0  0  0]
Q_values : [-1.7115932 -2.1202722  3.142397  -0.9480819]
state : [0 2 6 0 0 2 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 9 4 1 2 4 2 0 1 3 0 2
 0 0 4 5 4 0 0 0 0 2 1]
Q_values : [-1.3288559   1.6066549  -0.12676156 -2.0707145 ]
state : [0 2 0 0 0 2 0 2 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 1 2 3 1 2 4 6 2 0 1 7 0 0
 0 0 2 0 8 0 0 0 0 0 0]
Q_values : [-4.769677 -3.

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-5.054762  -4.5304084 -4.7159677 -4.44375  ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 3 2 2 2 4 2 2 3 3 2 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-11.340146 -13.375751 -13.582655 -11.950918]
state : [0 0 2 0 2 2 0 0 2 0 3 2 0 0 0 0 0 0 0 0 0 0 0 0 1 0 7 2 2 2 5 0 2 1 7 2 2
 1 0 2 0 0 1 1 1 2 0 0]
Q_values : [-1.4605365  1.6818798  1.332685  -2.7299354]
state : [ 0  0  0  0  2  2  0  0  0  0  6  2  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  3  2  4  4  4  0  0  0 10  2  0  0  0  4  0  7  0  0  0  0  0  1]
Q_values : [-2.87962   -0.8085054  2.3697598 -2.902063 ]
state : [0 0 2 0 0 4 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 1 6 4 1 4 4 0 0 1 3 2 2
 0 0 3 6 4 0 0 0 0 2 1]
Q_values : [-2.9258623 -2.217699  -1.5967234 -1.3420615]
state : [ 0  0  6  0  0  0  0  0  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  0 10  1  2  2  5  0  0  1  7  0  0  0  1  1  1

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-5.010467  -3.9377444 -4.5800414 -4.424393 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 3 2 2 2 4 2 2 3 3 2 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-10.665113 -10.014373  -8.661282 -10.300933]
state : [0 2 2 0 0 2 0 2 2 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 1 2 7 2 0 2 5 2 2 1 3 2 2
 0 0 2 2 0 2 0 0 1 2 0]
Q_values : [-0.50413305  1.9768608  -0.36201483  0.36495703]
state : [0 2 0 0 0 2 0 2 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 2 2 4 4 2 0 0 6 2 0
 0 0 3 0 8 0 0 0 0 0 1]
Q_values : [-5.289276  -4.2441535 -2.5052183 -2.8503923]
state : [0 2 2 0 0 4 0 2 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 3 6 4 1 4 4 2 0 1 3 2 2
 0 0 4 3 2 0 0 0 0 1 0]
Q_values : [-3.5926604 -2.520935  -2.521415  -1.6476558]
state : [ 0  2  6  0  0  0  0  2  0  0  3  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  2 10  1  2  2  6  2  0  1  7  0  0  0  1  2  0  0  0  0  3  0  0  0]
Q_values : [-4.069272

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-5.3530426 -4.8761396 -4.541785  -4.513396 ]
state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 3 2 2 2 4 2 2 3 3 2 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-11.340983 -12.169029  -9.766932 -12.227298]
state : [0 2 2 0 0 2 0 2 2 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 1 2 7 2 0 2 5 2 2 1 3 2 2
 0 0 2 2 0 2 0 0 1 2 0]
Q_values : [ 0.3367328   1.8083503  -0.69677323 -0.80258644]
state : [0 2 0 0 0 2 0 2 0 0 2 2 0 0 0 0 0 0 0 0 0 0 0 0 0 2 3 2 2 4 4 2 0 0 6 2 0
 0 0 3 0 8 0 0 0 0 0 1]
Q_values : [-5.8408365 -5.72272   -3.8066425 -3.607748 ]
state : [0 2 2 0 2 0 0 2 0 0 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 6 4 3 0 4 2 0 1 9 0 2
 0 1 4 0 0 0 0 3 0 0 0]
Q_values : [-6.69405103e-01  1.19268894e-04  3.81231475e+00 -2.12421393e+00]
state : [ 0  2  6  0  0  0  0  2  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  1  2 10  1  1  2  5  2  0  1  3  0  0  0  0  3  5  4  0  0  0  0  2  0]
Q

KeyboardInterrupt: 

In [36]:
model_action.save_weights("model_3DQN.weights.h5")
model_action.save("full_model.keras")

In [49]:
sumoCmd = [sumo_gui_bin, "-c",simulConfig,"--start", "--no-warnings"]

if traci.isLoaded():
    traci.close()
traci.start(sumoCmd)
lane_ids =  traci.lane.getIDList()
trafic_light_ids = traci.trafficlight.getIDList()
print(lane_ids)
state = np.array(get_state(lane_ids))
action=1
# print(state)
alpha=0.1
for step in range(130000): ## TO CHANGED
    nom_du_feu= traci.trafficlight.getIDList()[0]
    print()
    if step%2000 == 0:
        state=np.array(get_state(lane_ids))
        action = epsilon_greedy_policy(state,0)*2
        traci.trafficlight.setPhase(trafic_light_ids[0],action)
    traci.simulationStep()

traci.close()


 Retrying in 1 seconds
(':junction_0_0', ':junction_10_0', ':junction_11_0', ':junction_1_0', ':junction_2_0', ':junction_3_0', ':junction_4_0', ':junction_5_0', ':junction_6_0', ':junction_7_0', ':junction_8_0', ':junction_9_0', 'end1_junction_0', 'end1_junction_1', 'end1_junction_2', 'end2_junction_0', 'end2_junction_1', 'end2_junction_2', 'end3_junction_0', 'end3_junction_1', 'end3_junction_2', 'end4_junction_0', 'end4_junction_1', 'end4_junction_2', 'junction_end1_0', 'junction_end1_1', 'junction_end1_2', 'junction_end2_0', 'junction_end2_1', 'junction_end2_2', 'junction_end3_0', 'junction_end3_1', 'junction_end3_2', 'junction_end4_0', 'junction_end4_1', 'junction_end4_2')

state : [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0]
Q_values : [-4.452952  -4.45352   -4.8745112 -4.1132607]























































































































































FatalTraCIError: Connection closed by SUMO.